<a href="https://colab.research.google.com/github/dhruvilpateld/Financial-Report-Analyzer/blob/master/Flask_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install ngrok

In [2]:
pip install flask-ngrok

change directory to saved model folder

In [3]:
cd /content/drive/MyDrive/Flask-deploy-model

/content/drive/MyDrive/Flask-deploy-model


In [4]:
!pip install transformers

In [5]:
!pip install sumy

In [6]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

In [7]:
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')

model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')

In [8]:
!pip install PyPDF2

In [9]:
! pip install pyngrok

In [10]:
! ngrok authtoken 2OV11RctWtNAkLwZ031YcSRIVjf_3XZ69bp5CV4gwEiLhwBr

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


import required packages

In [11]:
from flask_ngrok import run_with_ngrok
from flask import Flask,request,render_template
from werkzeug.utils import secure_filename
import os
import numpy as np
from PyPDF2 import PdfReader
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
import nltk
import textwrap
nltk.download('punkt')
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Create Flask Application

In [12]:
from flask import Flask, request, render_template
from werkzeug.utils import secure_filename
import os

app = Flask(__name__)

run_with_ngrok(app)

# Set the path to where the PDF files will be saved
UPLOAD_FOLDER = 'pdf_uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Only allow PDF files to be uploaded
ALLOWED_EXTENSIONS = {'pdf'}

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload_file():
    # Check if the post request has the file part
    if 'file' not in request.files:
        return 'No file part'
    file = request.files['file']
    # If user does not select file, browser also
    # submit an empty part without filename
    if file.filename == '':
        return 'No selected file'
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))

        txt = """ """
        reader = PdfReader(os.path.join('pdf_uploads',filename))
        number_of_pages = len(reader.pages)
        for i in range(number_of_pages):
            page = reader.pages[i]
            text = page.extract_text()
            txt += text

        tokens = tokenizer.encode_plus(txt, add_special_tokens=False, return_tensors = 'pt')

        input_id_chunks = tokens['input_ids'][0].split(510)
        attention_mask_chunks = tokens['attention_mask'][0].split(510)

        chunksize = 512
        input_id_chunks = list(tokens['input_ids'][0].split(chunksize - 2))
        attention_mask_chunks = list(tokens['attention_mask'][0].split(chunksize - 2))

        for i in range(len(input_id_chunks)):
            input_id_chunks[i] = torch.cat([
                torch.tensor([101]), input_id_chunks[i], torch.tensor([102])
            ])

            attention_mask_chunks[i] = torch.cat([
                torch.tensor([1]), attention_mask_chunks[i], torch.tensor([1])
            ])

            pad_length = chunksize - input_id_chunks[i].shape[0]

            if pad_length > 0:
                input_id_chunks[i] = torch.cat([
                    input_id_chunks[i], torch.Tensor([0] * pad_length)
                ])
                attention_mask_chunks[i] = torch.cat([
                    attention_mask_chunks[i], torch.Tensor([0] * pad_length)
                ])

        input_ids = torch.stack(input_id_chunks)
        attention_mask = torch.stack(attention_mask_chunks)

        input_dict = {
            'input_ids' : input_ids.long(),
            'attention_mask' : attention_mask.int()
        }

        outputs = model(**input_dict)

        probabilities = torch.nn.functional.softmax(outputs[0], dim = -1 )

        mean_probabilities = probabilities.mean(dim = 0)
        output1=round(mean_probabilities[0].item()*100, 2)
        output2=round(mean_probabilities[2].item()*100, 2)
        output3=round(mean_probabilities[1].item()*100, 2)

        num_sentences=2
        parser = PlaintextParser.from_string(txt, Tokenizer("english"))
        summarizer = LexRankSummarizer()
        summary = summarizer(parser.document, num_sentences)
        summary_text = " ".join(str(sentence) for sentence in summary)
        summary_text = summary_text.replace("\n", " ")
        summary_text = summary_text.split(" o ")
        summary_text = "\n".join(summary_text)

        input_txt=txt
        summary = summary_text
        sentences = re.split(r"(?<=[!.?] •o )\s", summary)
        new_text = "\n".join(sentences)
        wrapped_text = textwrap.fill(new_text, width=140)


        # Render the success.html template and pass the output values as arguments
        return render_template('success.html', filename=filename, output1=output1, output2=output2, output3=output3, new_paragraph=wrapped_text)
    else:
        return 'Invalid file type'


if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://a884-34-135-68-132.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [30/Jul/2023 10:46:45] "GET / HTTP/1.1" 200 -
Token indices sequence length is longer than the specified maximum sequence length for this model (6308 > 512). Running this sequence through the model will result in indexing errors
INFO:werkzeug:127.0.0.1 - - [30/Jul/2023 10:47:30] "POST /upload HTTP/1.1" 200 -
